In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from time import sleep
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from datetime import date,timedelta,datetime

### Get all days from 1-1-2020 to current date

In [ ]:
d0 = date(2020, 1, 1) 
d1 = datetime.date(datetime.now())  #datetime.today() 
delta = d1 - d0

base = datetime.today() # Get the current date
date_list = [(base - timedelta(days=x)).strftime('%d/%m/%Y') for x in range(delta.days)]

In [ ]:
date_list = date_list[1:] # Remove the current day
len(date_list)

876

### Open the chrome browser and login

In [ ]:
#Using chromedriver to open browser which install automaticly
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# Open the browser with the link
browser.get("https://finance.vietstock.vn/ket-qua-giao-dich")
sleep(2)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\lenovo\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [ ]:
#Click the login button
browser.find_element(By.XPATH, "/html/body/div[2]/div[6]/div/div[2]/div[2]/a[3]").click()

In [ ]:
EMAIL = 'teon11223@gmail.com'
PASSWORD = '412001'
# Fill in login form and click the login button
email_field = browser.find_element(By.ID, "txtEmailLogin")
email_field.send_keys(EMAIL)
sleep(2)
password_field = browser.find_element(By.ID, "txtPassword")
password_field.send_keys(PASSWORD)
sleep(2)
browser.find_element(By.ID, "btnLoginAccount").click()
sleep(1)

### Crawl data

In [ ]:
def crawl_stock_price():
    page_source = BeautifulSoup(browser.page_source)
    rows = page_source.find_all('tr')[9:]
    crawled_data = []
    for tr in rows:
        lst_td = tr.find_all('td')
        crawled_data.append([lst_td[1].text,lst_td[2].text,lst_td[5].text])
    
    return crawled_data

In [ ]:
def crawl_100_symbol_day():
    data = crawl_stock_price()
    while len(data) != 100: # Get 100 stock symbols
        # Click the next button
        browser.find_element(By.ID, "btn-page-next").click()
        sleep(5)
        crawl_dt = crawl_stock_price()
        if crawl_dt not in data:
            data.extend(crawl_dt)
        sleep(2)
    return data

In [ ]:
data = crawl_100_symbol_day()
browser.find_element(By.XPATH, "//*[@id='view-tab']/li[1]/a").click()
sleep(2)

for day in date_list:
    # Scroll window to coordinates that get get needed element
    browser.execute_script('window.scrollTo(0,260);')
    sleep(1)
    # Send key to from date field
    from_date = browser.find_element(By.XPATH, "//*[@id='txtFromDate']/input")
    from_date.clear()
    from_date.send_keys(day)
    sleep(2)
    # Send key to date field
    to_date = browser.find_element(By.XPATH, "//*[@id='txtToDate']/input")
    to_date.clear()
    to_date.send_keys(day)
    sleep(2)
    # Click Xem button
    browser.find_element(By.XPATH, "//*[@id='trading-result']/div/div[1]/div[1]/div/div[2]/button").click()
    sleep(2)
    
    data.extend(crawl_100_symbol_day())
    browser.find_element(By.XPATH, "//*[@id='view-tab']/li[1]/a").click()
    sleep(2)

In [ ]:
len(data)

87700

In [ ]:
#Convert data list to dataframe
df = pd.DataFrame(data, columns=['Date','Symbols','Close Prices'])
df.head()

,Date,Symbols,Close Prices
0,27/05/2022,AAA,12.65
1,27/05/2022,AAM,12.40
2,27/05/2022,AAT,15.10
3,27/05/2022,ABR,19.50
4,27/05/2022,ABS,13.90


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87700 entries, 0 to 87699
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          87700 non-null  object
 1   Symbols       87700 non-null  object
 2   Close Prices  87700 non-null  object
dtypes: object(3)
memory usage: 2.0+ MB


In [ ]:
#Save dataframe to file .csv
df.to_csv('Crawl_Stock_Prices.csv')